In [1]:
import pandas as pd
import numpy as np

# Dados Abertos

In [2]:
import os

In [3]:
def load_data(path):
    dfs = []
    for data in os.listdir(path):
        dfs.append(pd.read_csv(os.path.join(path, data), sep=';'))
    concat = pd.concat(dfs)
    return concat
        

In [4]:
propo = load_data('dados/proposicoes')
temas = load_data('dados/proposicoes_temas')
autores = load_data('dados/proposicoes_autores')

/tmp/ipykernel_70328/142014999.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(os.path.join(path, data), sep=';'))
/tmp/ipykernel_70328/142014999.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(os.path.join(path, data), sep=';'))


In [5]:
propo_filter = propo[['id','ano','descricaoTipo','ementa','keywords','ultimoStatus_idTipoTramitacao','ultimoStatus_descricaoSituacao']]

In [6]:
autores = autores[['idProposicao', 'idDeputadoAutor', 'codTipoAutor', 'tipoAutor', 'nomeAutor', 'siglaPartidoAutor', 'ordemAssinatura', 'proponente']]

In [7]:
agregado = propo_filter.merge(autores, left_on='id', right_on='idProposicao')

In [8]:
agregado['idDeputadoAutor'] = agregado['idDeputadoAutor'].fillna(-1).astype(int)

In [9]:
agregado_dep = agregado[(agregado['codTipoAutor']==10000) | (agregado['codTipoAutor']==20000) | (agregado['nomeAutor'].str.contains('Senado Federal -', regex=True))]

In [10]:
print('Proposições totais:', len(agregado),'\nProposições de deputados e senadores:', len(agregado_dep))

Proposições totais: 279683 
Proposições de deputados e senadores: 264783


In [11]:
proponentes = agregado_dep[agregado_dep['proponente']==1]

In [12]:
contagem_parlamentares = agregado_dep.groupby(['idDeputadoAutor','nomeAutor','descricaoTipo'], as_index=False).size()
contagem_parlamentares_proponentes = proponentes.groupby(['idDeputadoAutor','nomeAutor','descricaoTipo'], as_index=False).size()

In [13]:
deputados = pd.read_csv('BQ/camara/deputados_legislatura_56.csv')

In [14]:
def parlamentares_ativos(df):
    result = df[(df['idDeputadoAutor'].isin(deputados['id']))
                                |
                (df['idDeputadoAutor']==-1)]
    return result

In [15]:
def add_parlamentar_sem_proposicao(df):
    # adicionando parlamentares que não tem proposição
    add_deputados = deputados[-deputados['id'].isin(df['idDeputadoAutor'])][['id', 'nome_parlamentar']]
    add_deputados.columns = ['idDeputadoAutor', 'nomeAutor']
    # criando dataframe com linhas repetidas contendo os tipos de proposição para cada parlamentar que não tem proposição
    dummy_df = pd.concat([pd.DataFrame({'descricaoTipo':df.descricaoTipo.unique()})]*len(add_deputados), ignore_index=True)
    # repete cada linha do dataframe add_deputados para cada tipo de proposição
    add_deputados= pd.DataFrame(np.repeat(add_deputados.values, len(df.descricaoTipo.unique()), axis=0), columns=add_deputados.columns)
    add_deputados['descricaoTipo'] = dummy_df['descricaoTipo']
    add_deputados['size'] = 0
    return pd.concat([df, add_deputados])

In [16]:
selecao_contagem_parlamentares = parlamentares_ativos(contagem_parlamentares)
selecao_contagem_parlamentares_proponentes = parlamentares_ativos(contagem_parlamentares_proponentes)

In [17]:
contagem_parlamentares = add_parlamentar_sem_proposicao(selecao_contagem_parlamentares)
contagem_parlamentares_proponentes = add_parlamentar_sem_proposicao(selecao_contagem_parlamentares_proponentes)

In [18]:
topicos_de_interesse = [
    'Projeto de Lei','Projeto de Lei Complementar', 'Projeto de Resolução', 'Proposta de Emenda à Constituição',
    'Projeto de Decreto Legislativo de Medida Provisória', 'Projeto de Decreto Legislativo de Sustação de Atos Normativos do Poder Executivo',
    'Requerimento de Audiência Pública', 'Requerimento de Convocação de Ministro de Estado no Plenário','Requerimento de Informação',
    'Indicação','Parecer do Relator','Emenda de Relator', 'Emenda de Plenário a Projeto com Urgência', 'Emenda de Plenário à MPV (Ato Conjunto 1/20)'
    ,'Emenda de Plenário', 'Emenda de Redação em Plenário', 'Emenda na Comissão', 'Emenda à PEC', 'Substitutivo'
                       ]

In [19]:
def get_mean_by_topic(df, topic):
    return (df[df.descricaoTipo==topic]['size'].sum()) / 513

def get_median_by_topic(df, topic):
    return df[df.descricaoTipo==topic]['size'].median()

def get_df_parlamentar(df, nome):
    dep = df[df.nomeAutor.str.contains(nome, regex=True)]
    dep = dep[dep['descricaoTipo'].isin(topicos_de_interesse)]
    dep['media_congresso'] = dep.descricaoTipo.apply(lambda x: get_mean_by_topic(df, x))
    dep['mediana_congresso'] = dep.descricaoTipo.apply(lambda x: get_median_by_topic(df, x))
    return dep

In [20]:
pd.set_option('display.max_colwidth', None)

In [21]:
tabata = get_df_parlamentar(contagem_parlamentares,'Tabata Amaral')
rigoni = get_df_parlamentar(contagem_parlamentares, 'Rigoni')
alessandro = get_df_parlamentar(contagem_parlamentares, 'Alessandro Vieira')
tabata_proponentes = get_df_parlamentar(contagem_parlamentares_proponentes, 'Tabata Amaral')
rigoni_proponentes = get_df_parlamentar(contagem_parlamentares_proponentes, 'Rigoni')
alessandro_proponentes = get_df_parlamentar(contagem_parlamentares_proponentes, 'Alessandro Vieira')

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

In [23]:
def save_df(df, proponente=False):
    nome = '_'.join(df.nomeAutor.iloc[0].split(' '))
    df['media_congresso'] = df['media_congresso'].round()
    df['mediana_congresso'] = df['mediana_congresso'].round()
    if proponente:
        df.to_excel(f'resultados/{nome}_proponentes.xlsx')
        return
    df.to_excel(f'resultados/{nome}_autores.xlsx')

In [24]:
save_df(tabata)
save_df(rigoni)
save_df(alessandro)

In [25]:
save_df(tabata_proponentes, proponente=True)
save_df(rigoni_proponentes, proponente=True)
save_df(alessandro_proponentes, proponente=True)

In [26]:

# for tipo in tabata.descricaoTipo:
#     f, ax = plt.subplots(figsize=(12, 8))
#     sns.set_theme(style="whitegrid")
#     sns.set_color_codes("pastel")
#     sns.barplot (data=tabata[tabata.descricaoTipo==tipo][['size','media_congresso','mediana_congresso']], palette='Blues_d', color='b')
#     plt.legend([f'{tabata[tabata.descricaoTipo==tipo].nomeAutor.iloc[0]}', 'Média Congresso', 'Mediana Congresso'])
#     plt.title(f'Tabata Amaral: {tipo} x Congresso')
#     plt.show()

Análise:
- Contabilizar proposições dos deputados do gabinete e comparar com restante do congresso
- Contabilizar Requerimentos/indicações
- Contabilizar Relatoria
- Contabilidzar Emendas
- Taxa de aprovação para cada ponto

Anotações:
- Investigar anos nulos (iguais a zero)
- Investigar processo de tramitação de proposições do legislativo
- 

In [27]:
autores.head()

,idProposicao,idDeputadoAutor,codTipoAutor,tipoAutor,nomeAutor,siglaPartidoAutor,ordemAssinatura,proponente
0,2325052,NaN,40000,Órgão do Poder Legislativo,Senado Federal - Eduardo Girão,PODEMOS,1,1
1,2324720,NaN,40000,Órgão do Poder Legislativo,Senado Federal - Eduardo Girão,PODEMOS,1,1
2,2323055,NaN,40000,Órgão do Poder Legislativo,Senado Federal,NaN,1,1
3,2319815,NaN,40000,Órgão do Poder Legislativo,Senado Federal - Jorge Kajuru,CIDADANIA,1,1
4,2318214,NaN,40000,Órgão do Poder Legislativo,Senado Federal,NaN,1,1
